# Start a Webserver using Serial

This example shows you how to start the webserver over serial connection

## Import packages

In [1]:
import sys
import os
import time

# Get the current working directory (the directory where the notebook is located)
notebook_dir = os.getcwd()

# Add the 'source' folder to the path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', 'source')))

# Now you can import from 'source'
from serial_controller import SerialController
from serial_command_sender import SerialCommandSender
from web_controller import WebController
from web_command_sender import WebCommandSender


## Initialize the Serial Command Sender and Connect to WiFi

Note: make sure nothing else is holding the serial line (for instance, a micropico VSC project). You may need to disconnect and reconnect the Pico's USB serial link.

After the webserver is started, serial commands can no longer be used. The webserver takes up the whole micropython thread.

In [ ]:
# Initialize the serial controller with the appropriate port. This must be done in every cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.
SERIAL_PORT = "COM6" #Find the serial port your RP Pico W is connected to, can use the MicroPico: Help > List Serial Ports in VSC's MicroPico package. Hit F1 to search for command in VSC
serial_controller = SerialController(port=SERIAL_PORT, debug=False)  # Adjust port as needed
serial_controller.connect()
# Initialize the command sender
serial_command_sender = SerialCommandSender(serial_controller)

# List available commands
print("Available commands:")
response = serial_command_sender.send_command('list_commands')
response_with_tabs = "\n".join("\t" + line for line in response.splitlines())
print(response_with_tabs) # Print the response with tabs to make it easier to read

# Set WiFi credentials
WIFI_SSID = "YOUR_WIFI_SSID"
WIFI_PASSWORD = "YOUR_WIFI_PASSWORD"
response = serial_command_sender.send_command("set_wifi_credentials",WIFI_SSID, WIFI_PASSWORD)
print(response)

# Connect to the internet (this may take up to 10 seconds)
response = serial_command_sender.send_command("connect_wifi")
print(response)


# Configure the Webserver (optional)
settings = {
    "port": 80, # Port for the webserver
    "debug": False # Level of printed information in micropython on pico
}
# Connect to the internet (this may take up to 10 seconds)
response = serial_command_sender.send_command("apply_webserver_settings", settings)
print(response)

# Start Webserver
response = serial_command_sender.send_command("start_webserver")
print(response)


serial_controller.close() #Serial controller must be closed in each cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.


## Initialize Web Command Sender and Turn an LED on and off

Use the webserver address printed above to connect to the RP Pico

In [ ]:
# Initialize the web controller with the base URL of the webserver
BASE_URL = "http://192.168.50.199:80" # Change to the IP address of your RP Pico W, see output above
web_controller = WebController(base_url=BASE_URL, debug=False)
# Initialize the command sender
web_command_sender = WebCommandSender(web_controller)

# Create a GPIO hardware, this case we'll use the LED
settings = {
    'pin_number': "LED", # Use the built-in LED
    'mode': 'OUT', # Set the pin as an output
    'value': 0, # Set the initial value to 0 (turn off the LED)
}
print(web_command_sender.send_command('create', 'gpio', settings, "test_gpio"))

# Start the GPIO hardware, this assigns to settings to an actual component on the board
print(web_command_sender.send_command('start', 'test_gpio'))

# Set GPIO value to 1 (turn on LED)
print(web_command_sender.send_command('apply_hardware_settings', 'test_gpio', {'value': 1}))

time.sleep(1)  # Wait for 1 second

# Set GPIO value back to 0 (turn off LED)
print(web_command_sender.send_command('apply_hardware_settings', 'test_gpio', {'value': 0}))

# Delete the test GPIO hardware
print(web_command_sender.send_command('delete', 'test_gpio'))

# Stop the webserver
print(web_command_sender.send_command('stop_webserver'))


## Configure Board to Start Webserver on power up

It would be quite useless to only be able to start the webserver when the serial connection is plugged in. So we store some settings in a config.json file on the board.

In [ ]:
# Initialize the serial controller with the appropriate port. This must be done in every cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.
serial_controller = SerialController(port=SERIAL_PORT, debug=False)  # Adjust port as needed
serial_controller.connect()
# Initialize the command sender
serial_command_sender = SerialCommandSender(serial_controller)

# Get the entire configuration
print("Current configuration:")
response = serial_command_sender.send_command('get_all_config')
print(response)

# Set the webserver to start on initialization 
print("Current configuration:")
response = serial_command_sender.send_command('set_config', 'webserver.start_on_init', True)
print(response)

# Save the current configuration
print("Saving current configuration...")
print(serial_command_sender.send_command('save_config')) # Now whenever the RP Pico W is powered on, the webserver will start automatically

# Get the entire configuration
print("New configuration:")
response = serial_command_sender.send_command('get_all_config')
print(response)

serial_controller.close() #Serial controller must be closed in each cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.
